# Didi data cleaning _ without POI

In this notebook, I present a rough protocol of my data clearning process. First, I cleaned 'order_data', 'weather_data' and 'traffic_data'. Then merge these three tables into a large DataFrame. For data obtained each day, I first save it into a individual csv file, then contact all of them if need to in the future. Target varible 'gaps' is obtained from 'order_data'. 'weather' and 'traffic' are used as relevant features. Following official problem statement, I also take 'gaps' in past 3 time slices into features of each data points. How to deal with 'POI' is still a problem.  

Detailed disccussions in data preprocessing are presented before each code cells. 

** If this procedure more or less works, change code in this ipynb into a python moduel for later use **

## Step 0 prepare to load in all training data files

Use `datetime` moduel to construct a list of date strings from **2016-01-01** to **2016-01-21**.  
Use `os` moduel to find local data pathes.

In [20]:
import datetime

def datelist(start, end):
    '''generate a list of date strings in required format'''
    start_date = datetime.date(*start)
    end_date = datetime.date(*end)

    result = []
    curr_date = start_date
    while curr_date != end_date:
        result.append("%04d-%02d-%02d" % (curr_date.year, curr_date.month, curr_date.day))
        curr_date += datetime.timedelta(1)
    result.append("%04d-%02d-%02d" % (curr_date.year, curr_date.month, curr_date.day))
    return result

#print datelist((2016, 7, 28), (2016, 8, 3))

In [21]:
import os

#print os.getcwd()
pwd = os.getcwd()
data_set_type = '//training_data'
folder_list = os.listdir(pwd+data_set_type)
print folder_list

['cluster_map', 'order_data', 'poi_data', 'traffic_data', 'weather_data']


In [24]:
date_list = datelist((2016, 1, 1), (2016, 1, 21))  ### generate time list of training data
print date_list

['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04', '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08', '2016-01-09', '2016-01-10', '2016-01-11', '2016-01-12', '2016-01-13', '2016-01-14', '2016-01-15', '2016-01-16', '2016-01-17', '2016-01-18', '2016-01-19', '2016-01-20', '2016-01-21']


## Step 1 generate a map between district_hash & district_id 

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

cluster_folder_location = pwd + data_set_type + '//cluster_map'
file_name = '//cluster_map'
col_name = ['district_hash','district_id']

id_map = pd.read_table(cluster_folder_location + file_name,names = col_name)
def hash2id(district_hash,id_map):
    location = (id_map.iloc[:,0] == district_hash)
    if sum(location):
        return int(id_map[location].iloc[0,1])
    else:
        return 'invalidhash'
    
#print hash2id('4725c39a5e5f4c188d382da3910b3f3f',id_map)
district_number = id_map['district_id'].max()
print district_number
print cluster_folder_location

66
C:\Users\sjtu\Documents\didi\public_data\public_data//training_data//cluster_map


## Step 2 generate a map between real time & time_slice number 1-144

In [5]:
import time

def time2slice(time_sample):
    '''Calculate which time slice a specific time record belongs to'''
    if len(time_sample) != 0:
        time_refer = time_sample[:-8]+'00:00:00'
        time_sample_struct = time.strptime(time_sample,'%Y-%m-%d %H:%M:%S')
        time_refer_struct = time.strptime(time_refer,'%Y-%m-%d %H:%M:%S')
        time_interval = time.mktime(time_sample_struct) - time.mktime(time_refer_struct)
        return int(np.ceil(time_interval/600.0))
    else:
        return np.nan

### generate a standard time series, from 1 to 144, for later use
time_standard = pd.DataFrame(data = np.arange(144)+1,columns = ['time'])

## Step 3 load order_data and turn it into required form

In [6]:
order_folder_location = pwd + data_set_type + '//order_data'
file_name = '//order_data_'+ date_list[0]
feature_name = ['order_id','driver_id','passenger_id','start_district_hash','dest_district_hash','Price','Time']

order_data = pd.read_table(order_folder_location + file_name,names = feature_name)
print 'data set read successfully!'
print len(order_data)

feature_col = list(order_data.columns)
#print len(feature_col)
#print feature_col
display(order_data.head(3))

data set read successfully!
501287


,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,Price,Time
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54
2,abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02


** Before counting gaps in each time slice, combine 'request' record from the same passenger id**

===

** According to discussion on forum, the problem has been simplified, count 'null' record directly **

In [7]:
order_data['no_response'] = order_data['driver_id'].isnull()
#display(order_data.head())

order_data_refined = order_data.copy()
order_data_refined = order_data_refined[:500]
useless_col = ['order_id','driver_id','dest_district_hash','Price']
order_data_refined.drop(useless_col,axis = 1,inplace = True)

order_data_refined['start_district_hash'] = order_data_refined['start_district_hash'].apply(hash2id,id_map = id_map)
order_data_refined['Time'] = order_data_refined['Time'].apply(time2slice)

#display(order_data_refined.head())
''''
# These lines are useless now.
order_data_fixed = order_data_refined['no_response'].groupby(
    [order_data_refined['start_district_hash'], order_data_refined['Time'],order_data_refined['passenger_id']]).min()
#print gaps.reset_index()
order_data_fixed = order_data_fixed.reset_index()  ###turn the groupby object into dataframe again

display(order_data_fixed.head(10))
'''
#gaps = order_data_refined['no_response'].groupby(
#    [order_data_refined['start_district_hash'], order_data_refined['Time']]).sum()
gaps = order_data_refined.groupby(['start_district_hash','Time']).sum()
### dataframe 'gaps' is exactly what to predict in the problem
gaps = gaps.reset_index()

gaps = gaps.sort_values(by = ['start_district_hash', 'Time'])   ###sort gaps by time and district_id

In [8]:
### change column name into ['district_id','time','gaps'],for sake of later merge
gaps.columns = ['district_id','time','gaps']
display(gaps.head())

,district_id,time,gaps
0,1,2,0.0
1,1,10,0.0
2,1,41,1.0
3,1,49,0.0
4,1,56,0.0


In [9]:
### Warning, the above process has a drawback ####
### If in a paticular time slice there is no passenger request, this time slice won't appear in above dataframe
### But according to problem statement, such a data point should be marked as 'gaps = 0'
### Fix this problem before further analysis

gaps_filled = pd.DataFrame(columns = gaps.columns)

for iid in np.arange(district_number)+1:
    gaps_tmp = gaps[gaps['district_id'] == iid]
    gaps_tmp_fixed = pd.merge(time_standard,gaps_tmp,how = 'outer', on = ['time']) ### add missing time-slice
    gaps_tmp_fixed = gaps_tmp_fixed.fillna({'district_id':iid,'gaps':0})
    gaps_filled = pd.concat([gaps_filled, gaps_tmp_fixed], ignore_index=True)
    
#gaps_1 = gaps_filled[gaps_filled['district_id'] == 5]
#print len(gaps_1)
#display(gaps_1.head(20))

gaps_final = gaps_filled ### use gaps_final as 'gaps' dataframe for later merge

## Step 4 load traffic data and change it into numerical features

In [10]:
traffic_data_location = pwd + data_set_type + '//traffic_data'
file_name = '//traffic_data_' + date_list[0]
feature_name = ['district_hash','tj_level_1','tj_level_2','tj_level_3','tj_level_4','tj_time']

traffic_data = pd.read_table(traffic_data_location + file_name,names = feature_name)
print 'data set read successfully!'
print len(traffic_data)

feature_col = list(traffic_data.columns)
print len(feature_col)
#print feature_col
display(traffic_data.head(3))

data set read successfully!
9289
6


,district_hash,tj_level_1,tj_level_2,tj_level_3,tj_level_4,tj_time
0,1ecbb52d73c522f184a6fc53128b1ea1,1:231,2:33,3:13,4:10,2016-01-01 23:30:22
1,1ecbb52d73c522f184a6fc53128b1ea1,1:305,2:69,3:13,4:16,2016-01-01 09:10:25
2,bf44d327f0232325c6d5280926d7b37d,1:579,2:87,3:28,4:16,2016-01-01 21:50:27


In [11]:
### map district_hash and tj_time into district_id
### turn tj_level into integers
### fill possible nan in tj_level by method = 'pad'

def level2num(rawlevel):
    if str(rawlevel) != 'nan':
        try:
            raw = rawlevel.split(':')
            return int(raw[1])
        except:
            print rawlevel
            return np.nan
    else:
        return np.nan

level_list = ['tj_level_1','tj_level_2','tj_level_3','tj_level_4']
for level in level_list:
    traffic_data[level] = traffic_data[level].apply(level2num)
    
traffic_data['district_hash'] = traffic_data['district_hash'].apply(hash2id,id_map = id_map)
traffic_data['tj_time'] = traffic_data['tj_time'].apply(time2slice)
traffic_data = traffic_data.fillna(method='pad')

#display(traffic_data.head(10))
display(traffic_data.describe())

### change column names into ['district_id','tj_level_1','tj_level_2','tj_level_3','tj_level_4','time'])
traffic_data.columns = ['district_id','tj_level_1','tj_level_2','tj_level_3','tj_level_4','time']
traffic_final = traffic_data  ### use traffic_final as 'traffic' dataframe for later merge
display(traffic_final.head())

,district_hash,tj_level_1,tj_level_2,tj_level_3,tj_level_4,tj_time
count,9289.000000,9289.000000,9289.000000,9289.000000,9289.000000,9289.000000
mean,33.196361,626.137905,125.200022,38.956508,26.674884,73.016363
std,19.026144,711.911224,198.321454,65.142351,41.727831,41.279923
min,1.000000,2.000000,0.000000,0.000000,0.000000,2.000000
25%,17.000000,164.000000,19.000000,6.000000,3.000000,37.000000
50%,33.000000,335.000000,51.000000,16.000000,11.000000,73.000000
75%,49.000000,802.000000,153.000000,46.000000,32.000000,109.000000
max,66.000000,4175.000000,1756.000000,714.000000,449.000000,144.000000


,district_id,tj_level_1,tj_level_2,tj_level_3,tj_level_4,time
0,66,231,33,13,10,142
1,66,305,69,13,16,56
2,64,579,87,28,16,132
3,64,242,28,10,9,39
4,64,471,35,15,17,6


## Step 5 load weather data and apply One-Hot-Encoding

In [12]:
weather_data_location = pwd +  data_set_type + '//weather_data'
file_name = '//weather_data_' + date_list[0]
feature_name = ['Time','Weather','temperature','PM2.5']

weather_data = pd.read_table(weather_data_location + file_name,names = feature_name)
print 'data set read successfully!'
print len(weather_data) ### if len(weather_data) != 288, indicates there are missing values

feature_col = list(weather_data.columns)
print len(feature_col)
#print feature_col
display(weather_data.head(3))

data set read successfully!
279
4


,Time,Weather,temperature,PM2.5
0,2016-01-01 00:00:28,1,4.0,177
1,2016-01-01 00:05:24,1,3.0,177
2,2016-01-01 00:10:08,1,3.0,177


In [13]:
weather_data['Time'] = weather_data['Time'].apply(time2slice)

#weather_data = weather_data.fillna(weather_data.mean()['temperature':'PM2.5'])
#weather_data['Weather'] = weather_data['Weather'].fillna(0)
#weather_data = weather_data.fillna(method='pad') ### fill missing values with pad??? maybe useless
display(weather_data.describe())

,Time,Weather,temperature,PM2.5
count,279.000000,279.000000,279.000000,279.000000
mean,71.103943,2.247312,6.344086,178.157706
std,41.525583,1.932248,3.502809,29.412415
min,1.000000,1.000000,2.000000,126.000000
25%,35.500000,1.000000,3.000000,153.500000
50%,70.000000,2.000000,7.000000,181.000000
75%,106.500000,2.000000,9.000000,205.000000
max,144.000000,9.000000,13.000000,232.000000


In [14]:
### sort weather_data by time
### remove repeated weather information
### fill missing values with weather data in previous data point

weather_data = weather_data.sort_values(by = ['Time'])
#weather_data = weather_data[1::2]
IsDuplicated = weather_data['Time'].duplicated()
weather_data = weather_data[IsDuplicated]
#display(weather_data.head(10))
print len(weather_data)

### change column names for future merge
weather_col = list(weather_data.columns)
weather_col[0] = 'time'

weather_data.columns = weather_col

weather_data_filled = pd.merge(time_standard,weather_data,how = 'left', on=['time'])
#display(weather_data_filled.describe())
weather_data_filled = weather_data_filled.fillna(method = 'pad')
display(weather_data_filled.describe())

137


,time,Weather,temperature,PM2.5
count,144.000000,144.000000,144.000000,144.00000
mean,72.500000,2.194444,6.430556,177.37500
std,41.713306,1.817852,3.507540,29.44424
min,1.000000,1.000000,2.000000,126.00000
25%,36.750000,1.000000,3.000000,153.00000
50%,72.500000,2.000000,8.000000,179.50000
75%,108.250000,2.000000,9.000000,197.50000
max,144.000000,8.000000,13.000000,232.00000


In [15]:
### apply one-hot-encoding on feature 'Weather'
def num2str(num):
    return str(int(num))

weather_data_filled['Weather'] = weather_data_filled['Weather'].apply(num2str)

#weather_data['Weather'].dtype == object
def OneHotEncoding(X):
    '''apply OHE to string features'''
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

weather_data_filled = OneHotEncoding(weather_data_filled)
weather_final = weather_data_filled ## use weather_final as 'weather' dataframe to merge
display(weather_final.head())

,time,Weather_1,Weather_2,Weather_3,Weather_8,temperature,PM2.5
0,1,1.0,0.0,0.0,0.0,3.0,177.0
1,2,1.0,0.0,0.0,0.0,3.0,177.0
2,3,1.0,0.0,0.0,0.0,3.0,177.0
3,4,1.0,0.0,0.0,0.0,3.0,177.0
4,5,1.0,0.0,0.0,0.0,3.0,177.0


## Step 5 merge gaps_final, traffic_final, weather_final into one dataframe
First Check if there still exist some mistakes, then merge, and save it as a csv file. This csv file will be used directly in knn model.

In [16]:
# dataframe to merge 1. gaps_final,traffic_final,weather_final
display(gaps_final.describe())
display(traffic_final.describe())
display(weather_final.describe())

,district_id,gaps,time
count,9504.000000,9504.000000,9504.000000
mean,33.500000,0.018939,72.500000
std,19.051374,0.176664,41.570404
min,1.000000,0.000000,1.000000
25%,17.000000,0.000000,36.750000
50%,33.500000,0.000000,72.500000
75%,50.000000,0.000000,108.250000
max,66.000000,4.000000,144.000000


,district_id,tj_level_1,tj_level_2,tj_level_3,tj_level_4,time
count,9289.000000,9289.000000,9289.000000,9289.000000,9289.000000,9289.000000
mean,33.196361,626.137905,125.200022,38.956508,26.674884,73.016363
std,19.026144,711.911224,198.321454,65.142351,41.727831,41.279923
min,1.000000,2.000000,0.000000,0.000000,0.000000,2.000000
25%,17.000000,164.000000,19.000000,6.000000,3.000000,37.000000
50%,33.000000,335.000000,51.000000,16.000000,11.000000,73.000000
75%,49.000000,802.000000,153.000000,46.000000,32.000000,109.000000
max,66.000000,4175.000000,1756.000000,714.000000,449.000000,144.000000


,time,Weather_1,Weather_2,Weather_3,Weather_8,temperature,PM2.5
count,144.000000,144.00000,144.000000,144.000000,144.000000,144.000000,144.00000
mean,72.500000,0.31250,0.597222,0.006944,0.083333,6.430556,177.37500
std,41.713306,0.46513,0.492169,0.083333,0.277350,3.507540,29.44424
min,1.000000,0.00000,0.000000,0.000000,0.000000,2.000000,126.00000
25%,36.750000,0.00000,0.000000,0.000000,0.000000,3.000000,153.00000
50%,72.500000,0.00000,1.000000,0.000000,0.000000,8.000000,179.50000
75%,108.250000,1.00000,1.000000,0.000000,0.000000,9.000000,197.50000
max,144.000000,1.00000,1.000000,1.000000,1.000000,13.000000,232.00000


In [17]:
### merge data
data_train = pd.merge(gaps_final, traffic_final, how='left', on=['district_id', 'time'])
data_train = pd.merge(data_train,weather_final,how = 'outer', on=['time'])

data_train = data_train.sort_values(by = ['district_id','time'])
display(data_train.head(10))

,district_id,gaps,time,tj_level_1,tj_level_2,tj_level_3,tj_level_4,Weather_1,Weather_2,Weather_3,Weather_8,temperature,PM2.5
0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,3.0,177.0
66,1.0,0.0,2.0,1399.0,318.0,102.0,94.0,1.0,0.0,0.0,0.0,3.0,177.0
132,1.0,0.0,3.0,1491.0,322.0,99.0,64.0,1.0,0.0,0.0,0.0,3.0,177.0
198,1.0,0.0,4.0,1490.0,287.0,98.0,78.0,1.0,0.0,0.0,0.0,3.0,177.0
264,1.0,0.0,5.0,1425.0,302.0,95.0,51.0,1.0,0.0,0.0,0.0,3.0,177.0
330,1.0,0.0,6.0,1327.0,313.0,94.0,66.0,1.0,0.0,0.0,0.0,3.0,177.0
396,1.0,0.0,7.0,1361.0,258.0,68.0,55.0,1.0,0.0,0.0,0.0,3.0,177.0
462,1.0,0.0,8.0,1395.0,280.0,97.0,69.0,1.0,0.0,0.0,0.0,3.0,177.0
528,1.0,0.0,9.0,1348.0,272.0,93.0,97.0,1.0,0.0,0.0,0.0,3.0,177.0
594,1.0,0.0,10.0,1417.0,236.0,93.0,53.0,1.0,0.0,0.0,0.0,3.0,177.0


In [18]:
### fill missing traffic level values in each district, then add previous 3 gaps as new features.
def add_nearby_gaps2features(data_tmp):
    data_tmp = data_tmp.reset_index(drop = True)
    gaps_pre_1 = data_tmp['gaps'].shift(1)
    gaps_pre_2 = data_tmp['gaps'].shift(2)
    gaps_pre_3 = data_tmp['gaps'].shift(3)
    data_tmp['gaps_pre_1'] = gaps_pre_1
    data_tmp['gaps_pre_2'] = gaps_pre_2
    data_tmp['gaps_pre_3'] = gaps_pre_3
    data_tmp= data_tmp.fillna({'gaps_pre_1':0,'gaps_pre_2':0,'gaps_pre_3':0})
    return data_tmp

data_train_fixed = pd.DataFrame(columns = data_train.columns)

for iid in np.arange(district_number)+1:
    data_tmp = data_train[data_train['district_id'] == iid]
    data_tmp = data_tmp.reset_index(drop = True)
    ### add 3 previous gaps to each points
    data_tmp = add_nearby_gaps2features(data_tmp)
    ### fillna in traffic level
    data_tmp = data_tmp.fillna(method = 'bfill')
    data_tmp = data_tmp.fillna(method = 'pad')
    data_tmp = data_tmp.fillna(data_tmp.mean())
    #if data_tmp['tj_level_1'].count() < 144:
    #    print iid
    #data_tmp = data_tmp.fillna(0)
    data_train_fixed = pd.concat([data_train_fixed, data_tmp], ignore_index=True)
    
data_train_fixed = data_train_fixed.fillna(data_train.mean())
display(data_train_fixed.describe())

,PM2.5,Weather_1,Weather_2,Weather_3,Weather_8,district_id,gaps,gaps_pre_1,gaps_pre_2,gaps_pre_3,temperature,time,tj_level_1,tj_level_2,tj_level_3,tj_level_4
count,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000,9504.000000
mean,177.375000,0.312500,0.597222,0.006944,0.083333,33.500000,0.018939,0.018939,0.018939,0.018939,6.430556,72.500000,626.082898,125.117719,38.930423,26.673630
std,29.343369,0.463537,0.490483,0.083048,0.276400,19.051374,0.176664,0.176664,0.176664,0.176664,3.495524,41.570404,706.699704,196.766661,64.642817,41.438284
min,126.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000
25%,153.000000,0.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,3.000000,36.750000,166.000000,20.000000,6.000000,4.000000
50%,179.500000,0.000000,1.000000,0.000000,0.000000,33.500000,0.000000,0.000000,0.000000,0.000000,8.000000,72.500000,341.000000,52.000000,16.000000,11.000000
75%,197.500000,1.000000,1.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,0.000000,9.000000,108.250000,793.250000,150.000000,45.000000,31.000000
max,232.000000,1.000000,1.000000,1.000000,1.000000,66.000000,4.000000,4.000000,4.000000,4.000000,13.000000,144.000000,4175.000000,1756.000000,714.000000,449.000000


In [19]:
### make new folders and save to csvfile
folder_name = 'merged_training_data_' + date_list[0]
new_path = os.path.join(pwd, folder_name)

if not os.path.isdir(new_path):

    os.makedirs(new_path)
    
data_train.to_csv(new_path + '\\test.csv',index = False)